**ส่วนนี้คือการทำ data cleaning (ท่านสามารถดูเพิ่มเติมได้ที่ 1_Data_Cleaning.ipynb)**

**สำหรับ Baseline Models จะเริ่มต่อท้ายหลังจบส่วนนี้**

In [1]:
import sys
sys.version

'3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import joblib

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
df = pd.read_parquet('./dataset/train_data.parquet')

#### ทำการเปลี่ยนชนิดข้อมูลให้ถูกประเภท

In [4]:
df['S_2'] = pd.to_datetime(df['S_2'], format='%Y-%m-%d')

In [5]:
category_cols = ['B_30','B_38','D_114','D_116','D_117','D_120','D_126','D_63','D_64','D_66','D_68']

for col in category_cols:
    df[col] = df[col].astype('category')

#### การจัดการกับค่าที่หายไป (Missing Value)

In [6]:
pd.options.display.max_rows = None # output unlimited rows

percent_missing = df.isnull().sum() * 100 / len(df)
df_missing_val = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})

In [7]:
df.drop(
    columns=df_missing_val.loc[
        df_missing_val.percent_missing > 50
    ].index, inplace=True, errors='ignore'
)

In [8]:
df.dropna(inplace=True)

#### สร้าง dummy variables

In [9]:
df = pd.get_dummies(df, columns=['D_63','D_64','D_68',
                                 'B_30','B_38',
                                 'D_114','D_116','D_117','D_120','D_126'], drop_first=True)
# drop_first=True in order to prevent the Multi-collinearity problem

**Baseline Models จะเริ่มต้นขึ้นต่อจากนี้**

---

# Baseline Models

สร้าง baseline models เพื่อเป็นตัวเปรียบเทียบประสิทธิภาพระหว่างก่อนและหลังการลดขนาดมิติข้อมูล

Baseline models ที่ใช้เพื่อเป็นตัวอ้างอิง คือ Classification Models ได้แก่ Logistic Regression, Decision Tree, Naïve Bayes, และ Random Forest

**(สามารถดูการเปรียบเทียบประสิทธิภาพระหว่างก่อนและหลังการลดขนาดมิติข้อมูลได้ที่ 5_Comparison.ipynb)**

In [10]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

#### เขียนฟังก์ชั่นสำหรับประเมินประสิทธิภาพ model และเก็บค่าไว้ทำกราฟเปรียบเทียบในภายหลัง:

In [11]:
acc = {}; f1 = {}; prec = {}; recl = {}

def my_eval(model,y_test,predicted):
    
    print(model,'Results')
    print('Accuracy: ', accuracy_score(y_test, predicted))
    print('F1 Score: ', f1_score(y_test, predicted))
    print('Precision: ', precision_score(y_test, predicted))
    print('Recall: ', recall_score(y_test, predicted))
    
    acc[model] = accuracy_score(y_test, predicted)
    f1[model] = f1_score(y_test, predicted)
    prec[model] = precision_score(y_test, predicted)
    recl[model] = recall_score(y_test, predicted)
    
    # declair as a global model to ratain the evalutation values along the notebook
    global df_model_compare
    df_model_compare = pd.DataFrame.from_dict(
        [acc,f1,prec,recl]
    ).rename(
        index={0:'Accuracy',1:'F1 Score',2:'Precision',3:'Recall'}
    )
    #display(df_model_compare)

## ทำการแบ่งข้อมูล train/test ด้วยอัตราส่วน 80:20

ทำการแบ่งข้อมูลเป็นสองชุด ได้แก่

1) Training Set เพื่อใช้ฝึกสอน model
2) Test Set เพื่อใช้ประเมิน model

นั่นคือ จะใช้ข้อมูลชุด Training Set ในขณะฝึกสอน model และ model จะไม่เคยเห็นข้อมูลชุด Test Set มาก่อน นี่ทำให้เราสามารถประเมิน model ได้อิงกับความเป็นจริงมากกว่า

ในการสร้าง models เพื่อทำนายว่าลูกค้าจะผิดนัดชำระหนี้หรือไม่ จะใช้ปัจจัย (features) ทุกตัวยกเว้น customer_ID เนื่องจากไม่ส่งผลต่อผลลัพธ์และ S_2 เนื่องจากไม่ทราบแน่ชัดว่าคือวันที่ของสิ่งใดและมีความหมายว่าอย่างไร (เนื่องจากชุดข้อมูลนี้ถูกทำให้เป็นแบบนิรนาม (anonymised) มาตั้งแต่ต้น)

กำหนดตัวแปร X (ตัวแปรอิสระ) และ y (ตัวแปรตาม):

In [12]:
X = df.drop(['target','customer_ID','S_2'],axis=1)
y = df['target']

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

---

## 1) Logistic Regression (Baseline)

In [14]:
import datetime

แสดง timestamp ก่อนรันอัลกอริทึม:

In [15]:
timer_start = datetime.datetime.now()
print(datetime.datetime.now().time())

22:26:46.782983


สร้าง (ฝึกสอน) model:

In [16]:
from sklearn.linear_model import LogisticRegression

logit = LogisticRegression(max_iter=1000, random_state=42)
logit.fit(X_train, y_train)

C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000, random_state=42)

แสดง timestamp หลังรันอัลกอริทึม:

In [17]:
timer_stop = datetime.datetime.now()
print(datetime.datetime.now().time())

22:31:38.985631


In [18]:
print('The time this algorithm utilised:',timer_stop - timer_start)

The time this algorithm utilised: 0:04:52.202648


**อัลกอริทึมนี้ใช้เวลาทำงานประมาณ 5 นาที**

สามารถเลือกที่จะบันทึก model ที่จะสร้างใหม่ หรือจะโหลดจาก model ที่สร้างไว้แล้วก็ได้:

In [19]:
# Save Model
joblib.dump(logit, './models/Logit_Baseline.pkl')

# Load Model
#logit = joblib.load('./models/Logit_Baseline.pkl')

['./models/Logit_Baseline.pkl']

ใช้โมเดลทำนาย test set:

In [20]:
logistic_predicted = logit.predict(X_test)

In [21]:
confusion_matrix(y_test, logistic_predicted)

array([[188662,  12595],
       [ 15997,  43070]], dtype=int64)

[ TN FP ]  
[ FN TP ]

TP (ทำนายว่า default; ความจริงคือ default)  
TN (ทำนายว่า non-default; ความจริงคือ non-default)  
FP (ทำนายว่า default; ความจริงคือ non-default)  
FN (ทำนายว่า non-default; ความจริงคือ default)

เรียกฟังก์ชั่นประเมินประสิทธิภาพ model และเก็บค่าไว้ทำกราฟเปรียบเทียบในภายหลัง:

In [22]:
my_eval('Logit',y_test,logistic_predicted)

Logit Results
Accuracy:  0.8901676372520397
F1 Score:  0.7507931527385558
Precision:  0.7737357405910357
Recall:  0.7291719572688642


---

## 2) Naïve Bayes (Baseline)

แสดง timestamp ก่อนรันอัลกอริทึม:

In [23]:
timer_start = datetime.datetime.now()
print(datetime.datetime.now().time())

22:31:39.850990


สร้าง (ฝึกสอน) model:

In [24]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
nb.fit(X_train, y_train)

GaussianNB()

แสดง timestamp หลังรันอัลกอริทึม:

In [25]:
timer_stop = datetime.datetime.now()
print(datetime.datetime.now().time())

22:31:43.323784


In [26]:
print('The time this algorithm utilised:',timer_stop - timer_start)

The time this algorithm utilised: 0:00:03.472794


**อัลกอริทึมนี้ใช้เวลาทำงานประมาณ 5 วินาที**

สามารถเลือกที่จะบันทึก model ที่จะสร้างใหม่ หรือจะโหลดจาก model ที่สร้างไว้แล้วก็ได้:

In [27]:
# Save Model
joblib.dump(nb, './models/NB_Baseline.pkl')

# Load Model
#nb = joblib.load('./models/NB_Baseline.pkl')

['./models/NB_Baseline.pkl']

ใช้โมเดลทำนาย test set:

In [28]:
nb_predicted = nb.predict(X_test)

In [29]:
confusion_matrix(y_test, nb_predicted)

array([[179390,  21867],
       [ 14733,  44334]], dtype=int64)

เรียกฟังก์ชั่นประเมินประสิทธิภาพ model และเก็บค่าไว้ทำกราฟเปรียบเทียบในภายหลัง:

In [30]:
my_eval('NB',y_test,nb_predicted)

NB Results
Accuracy:  0.859405971020728
F1 Score:  0.7078264201551873
Precision:  0.6696877690669325
Recall:  0.7505713850373305


---

## 3) Decision Tree (Baseline)

แสดง timestamp ก่อนรันอัลกอริทึม:

In [31]:
timer_start = datetime.datetime.now()
print(datetime.datetime.now().time())

22:31:44.805574


สร้าง (ฝึกสอน) model:

In [32]:
from sklearn.tree import DecisionTreeClassifier

dtree = DecisionTreeClassifier(random_state=42)
dtree.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

แสดง timestamp หลังรันอัลกอริทึม:

In [33]:
timer_stop = datetime.datetime.now()
print(datetime.datetime.now().time())

23:04:36.510354


In [34]:
print('The time this algorithm utilised:',timer_stop - timer_start)

The time this algorithm utilised: 0:32:51.704780


**อัลกอริทึมนี้ใช้เวลาทำงานประมาณ 33 นาที**

สามารถเลือกที่จะบันทึก model ที่จะสร้างใหม่ หรือจะโหลดจาก model ที่สร้างไว้แล้วก็ได้:

In [35]:
# Save Model
joblib.dump(dtree, './models/DT_Baseline.pkl')

# Load Model
#dtree = joblib.load('./models/DT_Baseline.pkl')

['./models/DT_Baseline.pkl']

ใช้โมเดลทำนาย test set:

In [36]:
dtree_predicted = dtree.predict(X_test)

In [37]:
confusion_matrix(y_test, dtree_predicted)

array([[183279,  17978],
       [ 17889,  41178]], dtype=int64)

เรียกฟังก์ชั่นประเมินประสิทธิภาพ model และเก็บค่าไว้ทำกราฟเปรียบเทียบในภายหลัง:

In [38]:
my_eval('DT',y_test,dtree_predicted)

DT Results
Accuracy:  0.8622216929672255
F1 Score:  0.6966157177537367
Precision:  0.6960916897694233
Recall:  0.6971405353242928


---

## 4) Random Forest (Baseline)

แสดง timestamp ก่อนรันอัลกอริทึม:

In [39]:
timer_start = datetime.datetime.now()
print(datetime.datetime.now().time())

23:04:37.488577


สร้าง (ฝึกสอน) model:

In [40]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

แสดง timestamp หลังรันอัลกอริทึม:

In [41]:
timer_stop = datetime.datetime.now()
print(datetime.datetime.now().time())

01:13:51.127182


In [42]:
print('The time this algorithm utilised:',timer_stop - timer_start)

The time this algorithm utilised: 2:09:13.638605


**อัลกอริทึมนี้ใช้เวลาทำงานประมาณ 2 ชั่วโมง 10 นาที**

สามารถเลือกที่จะบันทึก model ที่จะสร้างใหม่ หรือจะโหลดจาก model ที่สร้างไว้แล้วก็ได้:

In [43]:
# Save Model
joblib.dump(rf, './models/RF_Baseline.pkl')

# Load Model
#rf = joblib.load('./models/RF_Baseline.pkl')

['./models/RF_Baseline.pkl']

ใช้โมเดลทำนาย test set:

In [44]:
rf_predicted = rf.predict(X_test)

In [45]:
confusion_matrix(y_test, rf_predicted)

array([[190103,  11154],
       [ 12776,  46291]], dtype=int64)

เรียกฟังก์ชั่นประเมินประสิทธิภาพ model และเก็บค่าไว้ทำกราฟเปรียบเทียบในภายหลัง:

In [46]:
my_eval('RF',y_test,rf_predicted)

RF Results
Accuracy:  0.9080760897957929
F1 Score:  0.7946134303762703
Precision:  0.8058316650709374
Recall:  0.7837032522389829


---

In [47]:
display(df_model_compare)

,Logit,NB,DT,RF
Accuracy,0.890168,0.859406,0.862222,0.908076
F1 Score,0.750793,0.707826,0.696616,0.794613
Precision,0.773736,0.669688,0.696092,0.805832
Recall,0.729172,0.750571,0.697141,0.783703


บันทึกค่าประเมินประสิทธิภาพของแต่ละ model เพื่อใช้เปรียบเทียบใน notebook ชื่อ 5_Comparison.ipynb:

In [48]:
df_model_compare.to_csv('./eval_results/eval_results_baseline.csv')